In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
train=pd.read_csv('/Users/anshul/Documents/CMPE295A/ICPSR_35074/DS0001/training2.csv', sep='\t')
test=pd.read_csv('/Users/anshul/Documents/CMPE295A/ICPSR_35074/DS0001/test2.csv', sep='\t')
train['Type']='Train' #Create a flag for Train and Test Data set
test['Type']='Test'
fullData = pd.concat([train,test],axis=0) #Combined both Train and Test Data set

In [17]:
fullData.columns # This will show all the column names
fullData.head(10) # Show first 10 records of dataframe
fullData.describe() #You can look at summary of numerical fields by using describe() function
#train.head(2)

,CASEID,DISYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,EMPLOY,DETNLF,...,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,ALCDRUG,DSMCRIT,PSYPROB,HLTHINS,PRIMPAY,PROBABL
count,1.009800e+04,10098.0,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,...,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000,10098.000000
mean,1.854160e+04,2011.0,7.226084,1.344326,5.257873,4.783918,1.068528,2.770053,2.869875,-4.881858,...,0.005942,0.003862,0.003961,0.023668,2.119133,0.442068,-0.288374,-2.365716,-2.078035,0.990196
std,1.361284e+05,0.0,2.606565,0.475171,3.389631,1.325970,2.892388,1.480990,1.431926,6.120688,...,0.076857,0.062029,0.062816,0.152020,0.817730,7.740688,4.258162,6.161750,6.997690,0.098533
min,1.000000e+00,2011.0,2.000000,1.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,-9.000000,-9.000000,-9.000000,0.000000
25%,2.525250e+03,2011.0,5.000000,1.000000,4.000000,5.000000,1.000000,2.000000,3.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,-9.000000,1.000000,-9.000000,-9.000000,1.000000
50%,5.049500e+03,2011.0,7.000000,1.000000,5.000000,5.000000,1.000000,3.000000,3.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,2.000000,2.000000,1.000000,1.000000
75%,7.573750e+03,2011.0,9.000000,2.000000,5.000000,5.000000,2.000000,3.000000,4.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,7.000000,2.000000,4.000000,5.000000,1.000000
max,1.386292e+06,2011.0,12.000000,2.000000,23.000000,6.000000,4.000000,5.000000,4.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,3.000000,20.000000,2.000000,4.000000,9.000000,1.000000


In [19]:
fullData.isnull().any()

CASEID      False
DISYR       False
AGE         False
GENDER      False
RACE        False
ETHNIC      False
MARSTAT     False
EDUC        False
EMPLOY      False
DETNLF      False
PREG        False
VET         False
LIVARAG     False
PRIMINC     False
ARRESTS     False
STFIPS      False
CBSA        False
PMSA        False
REGION      False
DIVISION    False
SERVSETD    False
METHUSE     False
DAYWAIT     False
REASON      False
LOS         False
PSOURCE     False
DETCRIM     False
NOPRIOR     False
SUB1        False
ROUTE1      False
            ...  
ROUTE3      False
FREQ3       False
FRSTUSE3    False
NUMSUBS     False
IDU         False
ALCFLG      False
COKEFLG     False
MARFLG      False
HERFLG      False
METHFLG     False
OPSYNFLG    False
PCPFLG      False
HALLFLG     False
MTHAMFLG    False
AMPHFLG     False
STIMFLG     False
BENZFLG     False
TRNQFLG     False
BARBFLG     False
SEDHPFLG    False
INHFLG      False
OTCFLG      False
OTHERFLG    False
ALCDRUG     False
DSMCRIT   

In [20]:
ID_col = ['CASEID']
target_col = ["PROBABL"]
cat_cols = ['AGE','GENDER','RACE','ETHNIC','MARSTAT','EDUC','EMPLOY','PREG', 'VET','LIVARAG','PRIMINC','ARRESTS','DIVISION','REASON','REGION']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col))
other_col=['Type'] #Test and Train Data set identifier

In [21]:
num_cat_cols = num_cols+cat_cols # Combined numerical and Categorical variables

#Create a new variable for each variable having missing value with VariableName_NA 
# and flag missing value with 1 and other with 0

for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1 

In [22]:
#create label encoders for categorical features
for var in cat_cols:
 number = LabelEncoder()
 fullData[var] = number.fit_transform(fullData[var].astype('str'))

#Target variable is also a categorical so convert it
fullData["PROBABL"] = number.fit_transform(fullData["PROBABL"].astype('str'))

train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

/Users/anshul/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
